In [1]:
import os

In [2]:
%pwd

'd:\\Text Summarization Project\\Text_Summarization_END_to_END\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text Summarization Project\\Text_Summarization_END_to_END'

1. Update config.yaml %%TO BE DONE INSIDE config>config.yaml%%
2. Update params.yaml ##NO_PARAMS_IN_DATA_INGESTION##
3. Update entity
4. Update the configuration manager
5. update the components
6. update the pipeline
7. update the main.py
8. update the app.py

In [5]:
#update entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [10]:
#update configuration manager
class ConfigurationManager:
    #class having two instance attributes. one is 'config' and other is 'params'
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        #config and params value read from 'CONFIG_FILE_PATH' and 'PARAMS_FILE_PATH'
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        #create directories with artifacts_root as artifacts as mentioned in config>config.yaml
        create_directories([self.config.artifacts_root])

    
    #method with return type DataIngestionConfig defined above in the code
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        #config = config>config.yaml-->data_ingestion
        config = self.config.data_ingestion

        #create directories as root_dir in config>config.yaml
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [12]:
#update components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    #method to download file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    #method to extract zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
#create the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-22 17:35:04,322: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-22 17:35:04,346: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-22 17:35:04,350: INFO: common: created directory at: artifacts]
[2024-06-22 17:35:04,352: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-22 17:35:07,440: INFO: 4203238593: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2B2F:16BCE0:AD7A6:DD5D2:6676BDEF
Accept-Ranges: bytes
Date: Sat, 22 Jun 2024 12:05:05 GMT
Via: 1.1 varnish
X-Served-By: cache-del21725-DEL
X-Cache: MIS